In [1]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical


Using TensorFlow backend.


In [2]:
n_groups_per_col = 30
ind_categorical_data_train = pd.read_feather("../data/ind_categorical_data_train.feather")
ind_categorical_labels_train = pd.read_feather("../data/ind_categorical_labels_train.feather")

In [3]:
train_data = ind_categorical_data_train.to_numpy()

In [4]:
def fit_ind_categorical_model(train_data, train_labels):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(n_groups_per_col, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(train_data, train_labels, validation_split=0.2, batch_size=512, epochs=20, verbose=1)
    
    return (history, model)

In [5]:
# low
train_label_low = to_categorical(ind_categorical_labels_train.low.to_numpy(), n_groups_per_col)
(indipendent_categorical_low_history, indipendent_categorical_low_model) = fit_ind_categorical_model(train_data, train_label_low)
feather.write_dataframe(pd.DataFrame.from_dict(indipendent_categorical_low_history.history), "../data/indipendent_categorical_low_history.feather")
indipendent_categorical_low_model.save("../data/indipendent_categorical_low_model.hdf5")


Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
4342137/4342137 [==============================] - 32s 7us/step - loss: 3.2871 - accuracy: 0.0807 - val_loss: 3.1306 - val_accuracy: 0.0849
Epoch 2/20
4342137/4342137 [==============================] - 32s 7us/step - loss: 3.1559 - accuracy: 0.0923 - val_loss: 3.0785 - val_accuracy: 0.1083
Epoch 3/20
4342137/4342137 [==============================] - 33s 8us/step - loss: 3.1141 - accuracy: 0.0984 - val_loss: 3.1018 - val_accuracy: 0.0979
Epoch 4/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 3.1264 - accuracy: 0.0954 - val_loss: 3.1301 - val_accuracy: 0.0975
Epoch 5/20
4342137/4342137 [==============================] - 36s 8us/step - loss: 3.1231 - accuracy: 0.0964 - val_loss: 3.2002 - val_accuracy: 0.0792
Epoch 6/20
4342137/4342137 [==============================] - 33s 8us/step - loss: 3.0937 - accuracy: 0.1003 - val_loss: 3.0589 - val_accuracy: 0.1070
Epoch 7/20
4342137/4342137 [============

In [6]:
# high
train_label_high = to_categorical(ind_categorical_labels_train.high.to_numpy(), n_groups_per_col)
(indipendent_categorical_high_history, indipendent_categorical_high_model) = fit_ind_categorical_model(train_data, train_label_high)
feather.write_dataframe(pd.DataFrame.from_dict(indipendent_categorical_high_history.history), "../data/indipendent_categorical_high_history.feather")
indipendent_categorical_high_model.save("../data/indipendent_categorical_high_model.hdf5")

Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
4342137/4342137 [==============================] - 32s 7us/step - loss: 3.3066 - accuracy: 0.0788 - val_loss: 3.2053 - val_accuracy: 0.0750
Epoch 2/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 3.1526 - accuracy: 0.0928 - val_loss: 3.1419 - val_accuracy: 0.0884
Epoch 3/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.1325 - accuracy: 0.0947 - val_loss: 3.1116 - val_accuracy: 0.0909
Epoch 4/20
4342137/4342137 [==============================] - 32s 7us/step - loss: 3.1448 - accuracy: 0.0933 - val_loss: 3.2226 - val_accuracy: 0.0780
Epoch 5/20
4342137/4342137 [==============================] - 33s 8us/step - loss: 3.1506 - accuracy: 0.0931 - val_loss: 3.0978 - val_accuracy: 0.1042
Epoch 6/20
4342137/4342137 [==============================] - 33s 8us/step - loss: 3.1046 - accuracy: 0.0990 - val_loss: 3.0565 - val_accuracy: 0.1031
Epoch 7/20
4342137/4342137 [============

In [7]:
# close
train_label_close = to_categorical(ind_categorical_labels_train.close.to_numpy(), n_groups_per_col)
(indipendent_categorical_close_history, indipendent_categorical_close_model) = fit_ind_categorical_model(train_data, train_label_close)
feather.write_dataframe(pd.DataFrame.from_dict(indipendent_categorical_close_history.history), "../data/indipendent_categorical_close_history.feather")
indipendent_categorical_close_model.save("../data/indipendent_categorical_close_model.hdf5")

Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
4342137/4342137 [==============================] - 32s 7us/step - loss: 3.3870 - accuracy: 0.0642 - val_loss: 3.2446 - val_accuracy: 0.0793
Epoch 2/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2529 - accuracy: 0.0764 - val_loss: 3.2658 - val_accuracy: 0.0725
Epoch 3/20
4342137/4342137 [==============================] - 33s 8us/step - loss: 3.2431 - accuracy: 0.0781 - val_loss: 3.2229 - val_accuracy: 0.0817
Epoch 4/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2412 - accuracy: 0.0783 - val_loss: 3.2321 - val_accuracy: 0.0801
Epoch 5/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 3.2383 - accuracy: 0.0791 - val_loss: 3.2322 - val_accuracy: 0.0794
Epoch 6/20
4342137/4342137 [==============================] - 34s 8us/step - loss: 3.2365 - accuracy: 0.0791 - val_loss: 3.2790 - val_accuracy: 0.0697
Epoch 7/20
4342137/4342137 [============